In [ ]:
from ultralytics import YOLO, RTDETR

model = YOLO("../runs/segment/yolov8x-seg.pt_1280/weights/best.pt")

In [ ]:
models = glob.glob("../runs/*/*/weights/best.pt")
print(len(models))

In [ ]:
import glob
import pandas as pd

rows = []
models = glob.glob("../runs/*/*/weights/best.pt")
print(len(models))
for m in models:
    
    model_name, image_size = m.split("/")[-3].split("_")
    model_name = model_name.replace(".pt","")
    model = YOLO(m) if "yolo" in model_name.lower() else RTDETR(m)
    model_data={"model_name":model_name,"image_size":int(image_size)}
    metrics = model.val(imgsz=int(image_size),half=False,plots=True,batch=1,name=m.split("/")[-3])
    new_row = {**metrics.results_dict, **model_data}
    rows += [new_row]

df = pd.DataFrame(rows)
df.to_csv("yolo_results.csv")

In [ ]:
metrics.results_dict

In [ ]:
# Let's first read the uploaded file to understand its structure and contents.
import pandas as pd

# Load the CSV file
file_path = 'yolo_results_updated.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure and content
data.head()


In [ ]:
# Calculate F1 scores for Box (B) and Mask (M)
data['F1(B)'] = 2 * (data['metrics/precision(B)'] * data['metrics/recall(B)']) / (data['metrics/precision(B)'] + data['metrics/recall(B)'])
data['F1(M)'] = 2 * (data['metrics/precision(M)'] * data['metrics/recall(M)']) / (data['metrics/precision(M)'] + data['metrics/recall(M)'])

# Since we need to visualize these scores against each model along with the image size,
# Let's create a label combining model name and image size for better readability in the plots.
data['model_image_size'] = data['model_name'] + " (" + data['image_size'].astype(str) + ")"

# Now let's prepare the plots.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# Plot 1: All methods comparing the detection performance (F1 score for Box)
plt.figure(figsize=(12, 8))
plt.barh(data['model_image_size'], data['F1(B)'], color='skyblue')
plt.xlabel('F1 Score (Box)')
plt.ylabel('Model (Image Size)')
plt.title('Comparison of Detection Performance Across Different YOLO Models')
plt.tight_layout()
plt.show()

# Plot 2: Segmentation methods having (M) values (F1 score for Mask)
# Filtering data to include only those with valid Mask (M) metrics
segmentation_data = data[~data['F1(M)'].isna()]
plt.figure(figsize=(12, 8))
plt.barh(segmentation_data['model_image_size'], segmentation_data['F1(M)'], color='lightgreen')
plt.xlabel('F1 Score (Mask)')
plt.ylabel('Model (Image Size)')
plt.title('Comparison of Segmentation Performance Across Different YOLO Models')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
# To create a scatter plot grouped by method names, we first need to extract the method names from the model names.
# Assuming 'YoloV8', 'YoloV9', and 'RTDETR' are the distinct method names present in the 'model_name' column,
# let's extract these method names into a new column.

# This extraction is based on the assumption that the method names are prefixes of the model names.
data['method'] = data['model_name'].apply(lambda x: 'YoloV8' if 'yolov8' in x.lower() else ('YoloV9' if 'yolov9' in x.lower() else ('RTDETR' if 'rtdetr' in x.lower() else 'Other')))

# Now, let's plot F1 scores for Box and Mask (for those methods having Mask values) grouped by these method names.
plt.figure(figsize=(14, 8))

# We will plot F1(B) vs F1(M) and color-code by method.
# Note: Some methods may not have F1(M) values, so we'll handle that accordingly.
for method in data['method'].unique():
    subset = data[data['method'] == method]
    plt.scatter(subset['F1(B)'], subset['F1(M)'].fillna(0), label=method, s=100)  # Fill NaN F1(M) with 0 for plotting

plt.xlabel('F1 Score (Box)')
plt.ylabel('F1 Score (Mask)')
plt.title('Scatter Plot of F1 Scores for Box vs. Mask, Grouped by Method')
plt.legend(title='Method')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

file_path = 'yolo_results_updated.csv'
data = pd.read_csv(file_path)
# It seems there was a misunderstanding in sorting the YoloV8 models correctly.
# Let's correct the sorting logic specifically for YoloV8 models according to the specified sequence: N - S - M - L - X.
method_markers = {
    'YoloV8': 'o',  # Circle
    'YoloV9': 's',  # Square
    'RTDETR': '^',  # Triangle Up
}
# Adjusting the model_sorting_weight function to accurately sort YoloV8 models
def model_sorting_weight_corrected(model_name, image_size):
    # Define sorting weights for model sizes within each method
    size_weights = {
        'yolov8': {'n': 1, 's': 2, 'm': 3, 'l': 4, 'x': 5},
        'yolov9': {'c': 1, 'e': 2},
        'rtdetr': {'l': 1, 'x': 2}
    }
    # Extract method and size identifier
    method = model_name.split('-')[0].lower()[:6]
    size_identifier = model_name.replace("-seg","")[-1].lower()  # Get the last character as the size identifier
    # Assign weight based on method and size identifier
    weight = size_weights.get(method, {}).get(size_identifier, 99)  # Default weight if not found
    return (image_size,weight)

# Apply the corrected sorting function to each row to create a sorting key
data['sorting_key_corrected'] = data.apply(lambda row: model_sorting_weight_corrected(row['model_name'], row['image_size']), axis=1)

# Sort the data by this new sorting key
sorted_data_corrected = data.sort_values(by=['sorting_key_corrected', 'image_size'])

def map_image_size_to_marker_size(image_size):
    base_size = 200  # Base marker size
    return (image_size / max(data['image_size']) * base_size)

# Apply the marker size mapping
data['marker_size'] = data['image_size'].apply(map_image_size_to_marker_size)

# Re-generate the scatter plots with the data correctly sorted
plt.figure(figsize=(4, 4))
for method, marker in method_markers.items():
    subset = sorted_data_corrected[sorted_data_corrected['method'] == method]
    for image_size in [640,1280]:
        subsubset = subset[subset["image_size"] == image_size]
        #plt.scatter(subsubset['num_params_millions'], subsubset['F1(B)'], label=method, s=subsubset['marker_size'], marker=marker)
        plt.plot(subsubset['num_params_millions'], subsubset['metrics/mAP50-95(B)'], label=f"{method}({image_size})", marker=marker)
        

#plt.xticks(rotation=45, ha='right')
plt.xlabel('Model Params (Million)')
plt.ylabel('mAP50-95(B)')
plt.legend(title='Method(img size)')
plt.grid(True)
plt.tight_layout()

plt.savefig("det.pdf", format="pdf", bbox_inches="tight")
plt.show()

# For the segmentation plot, we need to make sure to use the correctly sorted subset that includes Mask scores
sorted_segmentation_data_corrected = sorted_data_corrected.dropna(subset=['F1(M)'])
plt.figure(figsize=(4, 4))
for method, marker in method_markers.items():
    subset = sorted_segmentation_data_corrected[sorted_segmentation_data_corrected['method'] == method]
    for image_size in [640,1280]:
        subsubset = subset[subset["image_size"] == image_size]
        #plt.scatter(subsubset['num_params_millions'], subsubset['F1(B)'], label=method, s=subsubset['marker_size'], marker=marker)
        plt.plot(subsubset['num_params_millions'], subsubset['metrics/mAP50-95(M)'], label=f"{method}({image_size})", marker=marker)
    break
        

#plt.xticks(rotation=0, ha='right')
plt.xlabel('Model Params (Million)')
plt.ylabel('mAP50-95(M)')
plt.legend(title='Method(img size)')
plt.grid(True)
plt.tight_layout()

plt.savefig("segm.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
len(data)

In [ ]:
model_params = {
    'yolov9c': 25.5,
    'yolov9e': 58.1,
    'rtdetr-l': 31.9,
    'rtdetr-x': 65.4,
    'yolov8x-seg': 68.3,
    'yolov8l-seg': 43.7,
    'yolov8m-seg': 25.9,
    'yolov8s-seg': 11.2,
    'yolov8n-seg': 3.2
}


# Add a new column to the data for the number of parameters based on the model name
data['num_params_millions'] = data['model_name'].map(model_params)

In [ ]:
import sys

if ".." not in sys.path:
    sys.path.insert(0,"..") 

from M18K.Data.Dataset import M18KDataset

ds = M18KDataset(root="train",transforms=None,train=True, depth=False)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg')
results = model.train(data=ds, epochs=100, imgsz=1280)

In [ ]:
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.utils import DEFAULT_CFG


class CustomTrainer(DetectionTrainer):
    def __init__(self, overrides=None, _callbacks=None):
        super().__init__(DEFAULT_CFG, {"task":"detect","data":"/home/azakeri/Mushroom/YOLO/datasets/data.yaml"}, _callbacks)
        self.args.task = "detect"
        self.args.data = "/home/azakeri/Mushroom/YOLO/datasets/data.yaml"
        self.data = "/home/azakeri/Mushroom/YOLO/datasets/data.yaml"

    # def build_dataset():
    #     return ds

trainer = CustomTrainer()
da = trainer.get_dataset({"train":"/home/azakeri/Mushroom/YOLO/datasets/data.yaml"})
#trainer.train()

In [ ]:
trainer.build_dataset()

In [ ]:
import glob
import shutil
import os

for mode in ["train","valid","test"]:
    images = glob.glob(f"../M18K/Data/{mode}/*.jpg")
    for i in images:
        filename = i.split("/")[-1][:-4]
        src = f"/home/azakeri/Mushroom/YOLO/datasets/{mode}/labels/{filename}.txt"
        if os.path.exists(src):
            
            shutil.copyfile(src,f"/home/azakeri/Mushroom/m18k/M18K/Data/{mode}/{filename}.txt")
        else:
            print(src)
        

In [ ]:
import os
import shutil

# Define your main directories
directories = ['/home/azakeri/Mushroom/m18k/M18K/Data/train', '/home/azakeri/Mushroom/m18k/M18K/Data/valid', '/home/azakeri/Mushroom/m18k/M18K/Data/test']

# Define the extensions for each file type
image_extensions = ['.jpg', '.jpeg', '.png']
label_extensions = ['.txt']
depth_extensions = ['.npy']

for dir in directories:
    # Create paths for the images, labels, and depth subdirectories
    images_dir = os.path.join(dir, 'images')
    labels_dir = os.path.join(dir, 'labels')
    depth_dir = os.path.join(dir, 'depth')
    
    # Create the subdirectories if they don't exist
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(depth_dir, exist_ok=True)
    
    # Iterate through each file in the main directory
    for filename in os.listdir(dir):
        file_path = os.path.join(dir, filename)
        
        # Move images
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            shutil.move(file_path, os.path.join(images_dir, filename))
        
        # Move labels
        elif any(filename.lower().endswith(ext) for ext in label_extensions):
            shutil.move(file_path, os.path.join(labels_dir, filename))
        
        # Move depth files
        elif any(filename.lower().endswith(ext) for ext in depth_extensions):
            shutil.move(file_path, os.path.join(depth_dir, filename))



WARNING ⚠️ imgsz=[720, 1280] must be multiple of max stride 32, updating to [736, 1280]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.69 GiB. GPU 0 has a total capacty of 31.74 GiB of which 733.88 MiB is free. Process 2677912 has 13.88 GiB memory in use. Including non-PyTorch memory, this process has 17.12 GiB memory in use. Of the allocated memory 12.88 GiB is allocated by PyTorch, and 3.06 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
import numpy as np
from PIL import Image
from skimage.measure import find_contours
from scipy.ndimage import zoom
from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

def create_coco_annotations(images, masks, bboxes, labels, scores, categories,seg):
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": categories
    }
    annotation_id = 0
    if seg:
        for image_id, (image_path, image_masks, bbox, label, score) in enumerate(
            zip(images, masks, bboxes, labels, scores), 0):
            with Image.open(image_path) as img:
                width, height = img.size
            coco_output["images"].append({
                "id": image_id,
                "file_name": image_path,
                "width": width,
                "height": height
            })
            for mask, box, lbl, sc in zip(image_masks, bbox, label, score):

                segmentation = []
                segmentation.append(mask.ravel().tolist())
                x1, y1, x2, y2 = box
                
                h, w = (x2 - x1), (y2 - y1)
                coco_output["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": lbl,
                    "bbox": [x1, y1, h, w],
                    "area": h * w,
                    "score": sc,
                    "segmentation": segmentation,
                    "iscrowd": 0
                })
                annotation_id += 1
    else:
        for image_id, (image_path, bbox, label, score) in enumerate(
            zip(images, bboxes, labels, scores), 0):
            with Image.open(image_path) as img:
                width, height = img.size
            coco_output["images"].append({
                "id": image_id,
                "file_name": image_path,
                "width": width,
                "height": height
            })
            for box, lbl, sc in zip(bbox, label, score):
                x1, y1, x2, y2 = box
                
                h, w = (x2 - x1), (y2 - y1)
                coco_output["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": lbl,
                    "bbox": [x1, y1, h, w],
                    "area": h * w,
                    "score": sc,
                    "iscrowd": 0
                })
                annotation_id += 1
    return coco_output

def getres(res,seg=True):
    images = []
    masks = []
    bboxes = []
    labels = []
    scores = []
    categories = [
        {
        "id": 0,
        "name": "Mushrooms",
        "supercategory": "none"
        },
        {
        "id": 1,
        "name": "BB",
        "supercategory": "Mushrooms"
        },
        {
        "id": 2,
        "name": "WB",
        "supercategory": "Mushrooms"
        }
    ]

    for result in res:  # Assuming 'results' is iterable; adjust based on actual structure
        image_path = result.path
        if seg:
            image_masks = result.masks.xy if result.masks and hasattr(result.masks, 'xy') else None
        bbox = result.boxes.xyxy.cpu().numpy()  # Convert to numpy array
        lbl = result.boxes.cls.cpu().numpy().astype(int) + 1
        sc = result.boxes.conf.cpu().numpy()
        
        # Append data for each image
        images.append(image_path)
        if seg:
            masks.append(image_masks)
        bboxes.append(bbox)
        labels.append(lbl)
        scores.append(sc)
        del result
        gc.collect()  # Invokes garbage collection
        if torch.cuda.is_available():
            torch.cuda.empty_cache()  # Clear CUDA cache

    coco_gt = COCO("../M18K/Data/test/_annotations.coco.json")
    categories = [{'id': 0, 'name': 'Mushrooms', 'supercategory': 'none'},
                    {'id': 1, 'name': 'BB', 'supercategory': 'Mushrooms'},
                    {'id': 2, 'name': 'WB', 'supercategory': 'Mushrooms'}]
    pred = create_coco_annotations(images,masks,bboxes,labels,scores,categories,seg)
    coco_pred = COCO()
    coco_pred.imgs = pred["images"]
    coco_pred.anns = pred["annotations"]
    coco_pred.cats = pred["categories"]
    coco_pred.dataset = pred
    coco_pred.createIndex()
    
    max_dets = [50, 100, 150]
    metrics = [
            f"Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets={max_dets[1]} ]",
            f"Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets={max_dets[2]} ]",
            f"Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets={max_dets[2]} ]",
            f"Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets={max_dets[2]} ]",
            f"Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets={max_dets[2]} ]",
            f"Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets={max_dets[2]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets={max_dets[0]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets={max_dets[1]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets={max_dets[2]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets={max_dets[2]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets={max_dets[2]} ]",
            f"Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets={max_dets[2]} ]"
        ]
    res = {}


    if seg:
        print("Segmentation Results: ")
        coco_eval = COCOeval(coco_gt, coco_pred, "segm")
        
        coco_eval.params.maxDets = max_dets
        coco_eval.params.useSegm=True
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()
        
        for m, v in zip(metrics, coco_eval.stats):
            res[f"SEG_{m}"] = v
    print("Detection Results: ")
    coco_eval = COCOeval(coco_gt, coco_pred, "bbox")
    coco_eval.params.maxDets = [50, 100, 150]
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    for m, v in zip(metrics, coco_eval.stats):
        res[f"DET_{m}"] = v
    return res

import glob
import pandas as pd
from ultralytics import YOLO, RTDETR
import gc
import torch

rows = []
models = glob.glob("../runs/*/*/weights/best.pt")
print(len(models))
for m in models:
    
    model_name, image_size = m.split("/")[-3].split("_")
    model_name = model_name.replace(".pt","")
    model = YOLO(m) if "yolo" in model_name.lower() else RTDETR(m)
    model_data={"model_name":model_name,"image_size":int(image_size)}
    images = ["../M18K/Data/test/images/4800_1680297637-021768_rgb_png_jpg.rf.0aa0c722ee22d6b6530d568d5abbe0ce.jpg","../M18K/Data/test/images/760_1680295931-1245968_rgb_png_jpg.rf.53f9680dc2bcddac3a59a1e7deeb9880.jpg","../M18K/Data/test/images/4770_1680297635-1662543_rgb_png_jpg.rf.6bd542c28628536e7ff7e8a4f0f46b8f.jpg","../M18K/Data/test/images/3840_1680297578-9777765_rgb_png_jpg.rf.5406d2b8f0f27022498cf0be14b4be1c.jpg","../M18K/Data/test/images/420_1680297424-24741_rgb_png_jpg.rf.4349f3d9e8ff6093516188ef8aac23ff.jpg","../M18K/Data/test/images/2600_1680296003-4456089_rgb_png_jpg.rf.003e8cea61fcaf3db7c07ba75df20e38.jpg","../M18K/Data/test/images/400_1680294882-263162_rgb_png_jpg.rf.9b1978dafd725d0563cfc88f7a6501f5.jpg","../M18K/Data/test/images/8600_1680295701-7331724_rgb_png_jpg.rf.09057b0a51e438a8c1cb40792e364980.jpg","../M18K/Data/test/images/3560_1680296373-8983989_rgb_png_jpg.rf.22bcd851cd249e7160a1903e6c15ecc2.jpg","../M18K/Data/test/images/1370_1680295955-3528156_rgb_png_jpg.rf.0960416ec226cca77774ac31ced244be.jpg","../M18K/Data/test/images/5240_1680297661-5673404_rgb_png_jpg.rf.1357ce943ea2b7ed7893e6b93dab4484.jpg","../M18K/Data/test/images/1750_1680297476-7382514_rgb_png_jpg.rf.5ff60c00aff05fe39919c4c01a130a27.jpg","../M18K/Data/test/images/2960_1680297524-4719222_rgb_png_jpg.rf.5a1401fd098a796bf34373e9488feb01.jpg","../M18K/Data/test/images/Dataset12_180_png_jpg.rf.a32d7abf1eb0a56bc138f3ee0898841e.jpg","../M18K/Data/test/images/1770_1680297477-542174_rgb_png_jpg.rf.b0304cfb426ac62a4dde2823c5f0adc0.jpg","../M18K/Data/test/images/10430_1680295776-692102_rgb_png_jpg.rf.901b9d1ce13da67c9cc2bd374b527ce7.jpg","../M18K/Data/test/images/570_1680297430-2199502_rgb_png_jpg.rf.9b68d49e5afe3a72ea6953b82de5e41f.jpg","../M18K/Data/test/images/8610_1680295702-1424067_rgb_png_jpg.rf.a61fbe90388348528893b3a889c24607.jpg","../M18K/Data/test/images/3900_1680297583-533494_rgb_png_jpg.rf.0be60d8fe9ba5468663b31fd56c1345a.jpg","../M18K/Data/test/images/2690_1680296338-7358935_rgb_png_jpg.rf.7c5947984218b63ef5e5d8b5d7a602ab.jpg","../M18K/Data/test/images/1230_1680296279-5587924_rgb_png_jpg.rf.7fb90c3ed80884fe29030f49a9276aa1.jpg","../M18K/Data/test/images/2500_1680297506-4388893_rgb_png_jpg.rf.0b652da8408c3c9a247c52b424613b21.jpg","../M18K/Data/test/images/5110_1680297655-7566438_rgb_png_jpg.rf.950dcc82a48308b2491a5dab6a54e778.jpg","../M18K/Data/test/images/7390_1680295652-589121_rgb_png_jpg.rf.18b4cd169ffef9f97296fc05ce8a1050.jpg","../M18K/Data/test/images/1970_1680296309-2808306_rgb_png_jpg.rf.93cfe92cabfd5be7397ba4d788bf6d6c.jpg","../M18K/Data/test/images/590_1680297430-9836905_rgb_png_jpg.rf.b4d573bc5702b6fa36ea67b2ac65037a.jpg","../M18K/Data/test/images/2170_1680297493-3877618_rgb_png_jpg.rf.b183da64cb4be1ec64eb6809b8634aa8.jpg","../M18K/Data/test/images/450_1680297425-431395_rgb_png_jpg.rf.b5b9ffed9278ef7f687c35a5b7e1f208.jpg","../M18K/Data/test/images/Dataset11_2160_png_jpg.rf.baf2a0c15318d5ac594351e116263336.jpg","../M18K/Data/test/images/440_1680297425-0440187_rgb_png_jpg.rf.cb3ebcfd0b9f967292e005f66cbe2530.jpg","../M18K/Data/test/images/390_1680297423-0478218_rgb_png_jpg.rf.d3aac8971731ac7531adb70934dab924.jpg","../M18K/Data/test/images/3050_1680297528-0589273_rgb_png_jpg.rf.bad1089998fd9e12f1f194199ad2fa1d.jpg","../M18K/Data/test/images/2450_1680296328-814072_rgb_png_jpg.rf.e1bf7b91c9eda34e7c983740d1e7e37d.jpg","../M18K/Data/test/images/400_1680297423-4511974_rgb_png_jpg.rf.b89e2964c933e8329e4ee066fc0b701d.jpg","../M18K/Data/test/images/Dataset11_7040_png_jpg.rf.db274efd2560239680b231b0fe2f67f3.jpg","../M18K/Data/test/images/8630_1680295702-9745076_rgb_png_jpg.rf.ee072b33a71fe2f4b9cd5a22eff18225.jpg","../M18K/Data/test/images/2850_1680297520-1448119_rgb_png_jpg.rf.ea3f38bf5e2f59e06da143c7333ed85b.jpg","../M18K/Data/test/images/9250_1680295728-3525634_rgb_png_jpg.rf.e84bac96c7328aecc32082ffa17a3c11.jpg","../M18K/Data/test/images/10510_1680295780-02445_rgb_png_jpg.rf.e057286315d21dd0d2c748a58bd2d64f.jpg","../M18K/Data/test/images/10770_1680295790-6968145_rgb_png_jpg.rf.e0935913a788d2c8f5c004acbf57c2eb.jpg","../M18K/Data/test/images/510_1680296250-1163669_rgb_png_jpg.rf.d6db686378e7bf164b8952b7f830f951.jpg","../M18K/Data/test/images/11160_1680295806-674861_rgb_png_jpg.rf.f6b9f407bc8a889ca7a7c5a8c83679c2.jpg"]
    ress = []
    for im in images:
        res = model.predict(im,imgsz=int(1280))
        ress += res
        del res,
        gc.collect()  # Invokes garbage collection
        if torch.cuda.is_available():
            torch.cuda.empty_cache()  # Clear CUDA cache
    res = getres(ress,"yolov8" in model_name)
    new_row = {**res, **model_data}
    rows += [new_row]
    print(rows)
    df = pd.DataFrame(rows)
    df.to_csv("yolo_results.csv")
    del model, res, ress, images
    gc.collect()  # Invokes garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # Clear CUDA cache

18

image 1/1 /home/azakeri/Mushroom/m18k/Demo/../M18K/Data/test/images/4800_1680297637-021768_rgb_png_jpg.rf.0aa0c722ee22d6b6530d568d5abbe0ce.jpg: 736x1280 1 BB, 12 WBs, 122.7ms
Speed: 23.8ms preprocess, 122.7ms inference, 3.1ms postprocess per image at shape (1, 3, 736, 1280)

image 1/1 /home/azakeri/Mushroom/m18k/Demo/../M18K/Data/test/images/760_1680295931-1245968_rgb_png_jpg.rf.53f9680dc2bcddac3a59a1e7deeb9880.jpg: 736x1280 56 BBs, 15.8ms
Speed: 6.9ms preprocess, 15.8ms inference, 3.2ms postprocess per image at shape (1, 3, 736, 1280)

image 1/1 /home/azakeri/Mushroom/m18k/Demo/../M18K/Data/test/images/4770_1680297635-1662543_rgb_png_jpg.rf.6bd542c28628536e7ff7e8a4f0f46b8f.jpg: 736x1280 2 BBs, 38 WBs, 8.9ms
Speed: 6.8ms preprocess, 8.9ms inference, 5.9ms postprocess per image at shape (1, 3, 736, 1280)

image 1/1 /home/azakeri/Mushroom/m18k/Demo/../M18K/Data/test/images/3840_1680297578-9777765_rgb_png_jpg.rf.5406d2b8f0f27022498cf0be14b4be1c.jpg: 736x1280 21 WBs, 9.0ms
Speed: 6.9ms

In [19]:
df.head()

,Unnamed: 0,Detection
0,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.808244
1,Average Precision (AP) @[ IoU=0.50 | are...,0.943583
2,Average Precision (AP) @[ IoU=0.75 | are...,0.905063
3,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.303108
4,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.731847


'yolov8'

In [23]:
csvs = glob.glob("../tests/*/results.csv")
j = 18
s = ""
for csv in csvs:
    df = pd.read_csv(csv)
    if "Segmentation" in df.columns:
        seg = df.Segmentation.to_list()
        det = df.Detection.to_list()
        s += f"{j},"
        j+=1
        for i in seg:
            s+=f"{i},"
        for i in det:
            s+=f"{i},"
        s+=csv.split("/")[2]+","+"1280,"
        s=s[:-1]+"\n"
    else:
        det = df.Detection.to_list()
        s += f"{j},,,,,,,,,,,,,"
        j+=1
        for i in det:
            s+=f"{i},"
        s+=csv.split("/")[2]+","+"1280,"
        s=s[:-1]+"\n"
    
s

'18,0.6081644108770865,0.8966995258431513,0.6616059964461345,0.1774815779448398,0.4482732653723166,0.8115582958110669,0.6215106602063631,0.6647215228261713,0.6652007560530084,0.3615384615384615,0.5535536648610739,0.8517577801116577,0.8838513210417558,0.9362410447304006,0.9184095295184488,0.4555464955794041,0.8097465490198448,0.978653170211447,0.8209093265344984,0.9119237035313036,0.9132415949051051,0.735042735042735,0.8672557143779931,0.9863578574982036,yolov8,1280\n19,,,,,,,,,,,,,0.8082444073142587,0.943583448291956,0.9050630793090926,0.3031079060747398,0.73184656584366,0.9329494386849504,0.7688410243389826,0.8436413438278005,0.8436413438278005,0.5837606837606837,0.784771655577805,0.9526629263169532,fasterrcnn_mobilenet_v3,1280\n20,,,,,,,,,,,,,0.8075443876039848,0.9476144575444452,0.8929721436961531,0.2943532828291896,0.7337688227393154,0.936870278291402,0.7672995613199896,0.8422030270331267,0.8422030270331267,0.5811965811965811,0.7848874657414158,0.9535742910839644,fasterrcnn_efficie

In [11]:
df

,Unnamed: 0,Segmentation,Detection
0,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.608164,0.883851
1,Average Precision (AP) @[ IoU=0.50 | are...,0.896700,0.936241
2,Average Precision (AP) @[ IoU=0.75 | are...,0.661606,0.918410
3,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.177482,0.455546
4,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.448273,0.809747
5,Average Precision (AP) @[ IoU=0.50:0.95 | are...,0.811558,0.978653
6,Average Recall (AR) @[ IoU=0.50:0.95 | are...,0.621511,0.820909
7,Average Recall (AR) @[ IoU=0.50:0.95 | are...,0.664722,0.911924
8,Average Recall (AR) @[ IoU=0.50:0.95 | are...,0.665201,0.913242
9,Average Recall (AR) @[ IoU=0.50:0.95 | are...,0.361538,0.735043


In [6]:
model_name, image_size = m.split("/")[-3].split("_")
model_name = model_name.replace(".pt","")
model = YOLO(m) if "yolo" in model_name.lower() else RTDETR(m)
model_data={"model_name":model_name,"image_size":int(image_size)}
images = ["../M18K/Data/test/images/4800_1680297637-021768_rgb_png_jpg.rf.0aa0c722ee22d6b6530d568d5abbe0ce.jpg","../M18K/Data/test/images/760_1680295931-1245968_rgb_png_jpg.rf.53f9680dc2bcddac3a59a1e7deeb9880.jpg","../M18K/Data/test/images/4770_1680297635-1662543_rgb_png_jpg.rf.6bd542c28628536e7ff7e8a4f0f46b8f.jpg","../M18K/Data/test/images/3840_1680297578-9777765_rgb_png_jpg.rf.5406d2b8f0f27022498cf0be14b4be1c.jpg","../M18K/Data/test/images/420_1680297424-24741_rgb_png_jpg.rf.4349f3d9e8ff6093516188ef8aac23ff.jpg","../M18K/Data/test/images/2600_1680296003-4456089_rgb_png_jpg.rf.003e8cea61fcaf3db7c07ba75df20e38.jpg","../M18K/Data/test/images/400_1680294882-263162_rgb_png_jpg.rf.9b1978dafd725d0563cfc88f7a6501f5.jpg","../M18K/Data/test/images/8600_1680295701-7331724_rgb_png_jpg.rf.09057b0a51e438a8c1cb40792e364980.jpg","../M18K/Data/test/images/3560_1680296373-8983989_rgb_png_jpg.rf.22bcd851cd249e7160a1903e6c15ecc2.jpg","../M18K/Data/test/images/1370_1680295955-3528156_rgb_png_jpg.rf.0960416ec226cca77774ac31ced244be.jpg","../M18K/Data/test/images/5240_1680297661-5673404_rgb_png_jpg.rf.1357ce943ea2b7ed7893e6b93dab4484.jpg","../M18K/Data/test/images/1750_1680297476-7382514_rgb_png_jpg.rf.5ff60c00aff05fe39919c4c01a130a27.jpg","../M18K/Data/test/images/2960_1680297524-4719222_rgb_png_jpg.rf.5a1401fd098a796bf34373e9488feb01.jpg","../M18K/Data/test/images/Dataset12_180_png_jpg.rf.a32d7abf1eb0a56bc138f3ee0898841e.jpg","../M18K/Data/test/images/1770_1680297477-542174_rgb_png_jpg.rf.b0304cfb426ac62a4dde2823c5f0adc0.jpg","../M18K/Data/test/images/10430_1680295776-692102_rgb_png_jpg.rf.901b9d1ce13da67c9cc2bd374b527ce7.jpg","../M18K/Data/test/images/570_1680297430-2199502_rgb_png_jpg.rf.9b68d49e5afe3a72ea6953b82de5e41f.jpg","../M18K/Data/test/images/8610_1680295702-1424067_rgb_png_jpg.rf.a61fbe90388348528893b3a889c24607.jpg","../M18K/Data/test/images/3900_1680297583-533494_rgb_png_jpg.rf.0be60d8fe9ba5468663b31fd56c1345a.jpg","../M18K/Data/test/images/2690_1680296338-7358935_rgb_png_jpg.rf.7c5947984218b63ef5e5d8b5d7a602ab.jpg","../M18K/Data/test/images/1230_1680296279-5587924_rgb_png_jpg.rf.7fb90c3ed80884fe29030f49a9276aa1.jpg","../M18K/Data/test/images/2500_1680297506-4388893_rgb_png_jpg.rf.0b652da8408c3c9a247c52b424613b21.jpg","../M18K/Data/test/images/5110_1680297655-7566438_rgb_png_jpg.rf.950dcc82a48308b2491a5dab6a54e778.jpg","../M18K/Data/test/images/7390_1680295652-589121_rgb_png_jpg.rf.18b4cd169ffef9f97296fc05ce8a1050.jpg","../M18K/Data/test/images/1970_1680296309-2808306_rgb_png_jpg.rf.93cfe92cabfd5be7397ba4d788bf6d6c.jpg","../M18K/Data/test/images/590_1680297430-9836905_rgb_png_jpg.rf.b4d573bc5702b6fa36ea67b2ac65037a.jpg","../M18K/Data/test/images/2170_1680297493-3877618_rgb_png_jpg.rf.b183da64cb4be1ec64eb6809b8634aa8.jpg","../M18K/Data/test/images/450_1680297425-431395_rgb_png_jpg.rf.b5b9ffed9278ef7f687c35a5b7e1f208.jpg","../M18K/Data/test/images/Dataset11_2160_png_jpg.rf.baf2a0c15318d5ac594351e116263336.jpg","../M18K/Data/test/images/440_1680297425-0440187_rgb_png_jpg.rf.cb3ebcfd0b9f967292e005f66cbe2530.jpg","../M18K/Data/test/images/390_1680297423-0478218_rgb_png_jpg.rf.d3aac8971731ac7531adb70934dab924.jpg","../M18K/Data/test/images/3050_1680297528-0589273_rgb_png_jpg.rf.bad1089998fd9e12f1f194199ad2fa1d.jpg","../M18K/Data/test/images/2450_1680296328-814072_rgb_png_jpg.rf.e1bf7b91c9eda34e7c983740d1e7e37d.jpg","../M18K/Data/test/images/400_1680297423-4511974_rgb_png_jpg.rf.b89e2964c933e8329e4ee066fc0b701d.jpg","../M18K/Data/test/images/Dataset11_7040_png_jpg.rf.db274efd2560239680b231b0fe2f67f3.jpg","../M18K/Data/test/images/8630_1680295702-9745076_rgb_png_jpg.rf.ee072b33a71fe2f4b9cd5a22eff18225.jpg","../M18K/Data/test/images/2850_1680297520-1448119_rgb_png_jpg.rf.ea3f38bf5e2f59e06da143c7333ed85b.jpg","../M18K/Data/test/images/9250_1680295728-3525634_rgb_png_jpg.rf.e84bac96c7328aecc32082ffa17a3c11.jpg","../M18K/Data/test/images/10510_1680295780-02445_rgb_png_jpg.rf.e057286315d21dd0d2c748a58bd2d64f.jpg","../M18K/Data/test/images/10770_1680295790-6968145_rgb_png_jpg.rf.e0935913a788d2c8f5c004acbf57c2eb.jpg","../M18K/Data/test/images/510_1680296250-1163669_rgb_png_jpg.rf.d6db686378e7bf164b8952b7f830f951.jpg","../M18K/Data/test/images/11160_1680295806-674861_rgb_png_jpg.rf.f6b9f407bc8a889ca7a7c5a8c83679c2.jpg"]
res = model.predict(images,imgsz=int(1280))
res = getres(res,"yolov8" in model_name)
new_row = {**res, **model_data}
rows += [new_row]
print(rows)
df = pd.DataFrame(rows)
df.to_csv("yolo_results_3.csv")
del model, res, images
gc.collect()  # Invokes garbage collection
if torch.cuda.is_available():
    torch.cuda.empty_cache()  # Clear CUDA cache

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.35 GiB. GPU 0 has a total capacty of 31.74 GiB of which 149.88 MiB is free. Process 2677912 has 13.88 GiB memory in use. Including non-PyTorch memory, this process has 17.70 GiB memory in use. Of the allocated memory 12.59 GiB is allocated by PyTorch, and 3.92 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
model = YOLO('../runs/detect/yolov9c.pt_640/weights/best.pt')
res = model("../M18K/Data/test/images/4800_1680297637-021768_rgb_png_jpg.rf.0aa0c722ee22d6b6530d568d5abbe0ce.jpg")



image 1/1 /home/azakeri/Mushroom/m18k/Demo/../M18K/Data/test/images/4800_1680297637-021768_rgb_png_jpg.rf.0aa0c722ee22d6b6530d568d5abbe0ce.jpg: 384x640 8 WBs, 14.5ms
Speed: 1.6ms preprocess, 14.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
conf: tensor([0.9312, 0.9273, 0.9111, 0.9060, 0.9040, 0.8891, 0.8264, 0.7497], device='cuda:0')
data: tensor([[0.0000e+00, 3.3799e+01, 8.3691e+01, 1.5694e+02, 9.3116e-01, 1.0000e+00],
        [1.1218e+03, 4.2590e+02, 1.2345e+03, 5.1928e+02, 9.2729e-01, 1.0000e+00],
        [3.0590e+02, 1.8845e+02, 3.9937e+02, 2.7990e+02, 9.1115e-01, 1.0000e+00],
        [1.1931e+03, 2.4960e+02, 1.2792e+03, 3.3260e+02, 9.0596e-01, 1.0000e+00],
        [1.2074e+03, 3.7327e+02, 1.2791e+03, 4.5679e+02, 9.0397e-01, 1.0000e+00],
        [7.2124e+02, 4.3853e-01, 8.2136e+02, 4.6046e+01, 8.8906e-01, 1.0000e+00],
        [1.2501e+03, 1.4055e+02, 1.2795e+03, 2.5055e+02, 8.2640e-01, 1.0000e+00],
        [1.0826e+03, 5.0012e+02, 1.1466e+03, 5.5815e+02, 7.4970e-01, 1.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([8, 6])
xywh: tensor([[  41.8456,   

In [84]:
import torch
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import cv2
naa = 0
for image, l, m, b in zip(images, labels, masks,bboxes):
    name = f"{naa}"
    naa+=1
    #img = t["img_org"]
    image = cv2.imread(image)
    colors = [(31, 119, 180) if x == 2 else (255, 127, 15) for x in l]
    m = np.sum(m,axis=0) > 0.5
    visualized = draw_bounding_boxes(torch.Tensor(image).type(torch.uint8).permute((2,0,1)), boxes=torch.Tensor(b), colors=colors, width=2)
    visualized = draw_segmentation_masks(visualized, masks=torch.tensor(m), alpha=0.6, colors=colors)
    visualized = visualized.permute(1, 2, 0).numpy().astype(np.uint8)
    #path = f"tests/maskrcnn_{self.backbone}" + ("_RGBD" if self.depth else "") + "/visualizations/"
    #os.makedirs(path, exist_ok=True)
    cv2.imwrite(f"{name}.jpg", cv2.cvtColor(visualized, cv2.COLOR_BGR2RGB))

In [13]:
a = []
for r in results:
    a += list(r.boxes.cls.cpu().numpy())

from collections import Counter
Counter(a)

Counter({0.0: 1420, 1.0: 490})

In [31]:
a = []
for l in labels:
    a += list(l)

from collections import Counter
Counter(a)

Counter({1: 1420, 2: 490})

In [19]:
np.unique(results[0].masks.data.cpu().numpy())

array([          0,           1], dtype=float32)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
creating index...
index created!
Segmentation Results: 
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.35s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.433
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=150 ] = 0.796
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=150 ] = 0.459
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=150 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=150 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=150 ] = 0.694
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 50 ] = 0.445
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.452
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=150 ] = 0.452
 Average R

In [ ]:
import numpy as np
import torch
def traverse_attributes(obj, parent_name=''):
    """
    Recursively traverse the attributes of a Python object,
    printing the attribute name, type, and shape (if applicable).

    Parameters:
    obj (any): The object to traverse.
    parent_name (str): String to hold the name of the parent attribute for nested attributes.
    """
    if not hasattr(obj, '__dict__') and not isinstance(obj, (list, tuple, dict)):
        return  # Base case: Not an object with attributes or a collection

    # For dict, list, and tuple, we handle them specifically to navigate through their elements
    if isinstance(obj, dict):
        for key, value in obj.items():
            prop_name = f"{parent_name}[{key}]" if parent_name else str(key)
            print_attribute(value, prop_name)
    elif isinstance(obj, (list, tuple)):
        for index, value in enumerate(obj):
            prop_name = f"{parent_name}[{index}]" if parent_name else str(index)
            print_attribute(value, prop_name)
    else:
        attributes = [attr for attr in dir(obj) if not attr.startswith('_')]
        for attr in attributes:
            value = getattr(obj, attr, None)
            prop_name = f"{parent_name}.{attr}" if parent_name else attr
            print_attribute(value, prop_name)

def print_attribute(value, prop_name):
    if isinstance(value, np.ndarray):
        print(f"{prop_name} (Type: {type(value)}, Shape: {value.shape})")
    elif isinstance(value, torch.Tensor):
        print(f"{prop_name} (Type: {type(value)}, Shape: {value.size()})")
    else:
        print(f"{prop_name} (Type: {type(value)})")
        # Recursively traverse if this attribute is also an object with attributes or a collection
        traverse_attributes(value, prop_name)

traverse_attributes(results[0])

In [ ]:
results[0].boxes.conf